In [0]:
-- Synthetic dataset (as JSON string) in a temp view
CREATE OR REPLACE TEMP VIEW v_payload_json AS
SELECT
  to_json(
    named_struct(
      'user',
      named_struct(
        'id',
        concat('u', cast(rand(3) * 10000000 as int)),
        'country',
        element_at(
          array('US', 'PL', 'IT', 'DE', 'KR', 'JP', 'BR', 'IN', 'GB', 'FR'),
          cast(rand(2) * 10 as int) + 1
        ),
        'age',
        cast(rand(4) * 70 as int)
      ),
      'metrics',
      named_struct('clicks', cast(rand(5) * 100 as int), 'spent', round(rand(6) * 1000, 2)),
      'flags',
      named_struct('is_paid', rand(7) < 0.35, 'is_new', rand(8) < 0.5),
      'items',
      transform(
        sequence(1, greatest(1, cast(rand(9) * 5 as int))),
        i -> named_struct(
          'sku',
          concat('sku_', id, '_', i),
          'price',
          round(rand(10) * 200, 2),
          'qty',
          cast(rand(11) * 5 as int)
        )
      ),
      'extra',
      map(
        'ua',
        element_at(array('Chrome', 'Safari', 'Firefox', 'Edge'), cast(rand(12) * 4 as int) + 1),
        'campaign',
        element_at(array('A', 'B', 'C', 'D'), cast(rand(13) * 4 as int) + 1)
      )
    )
  ) AS payload_json
FROM
  range(10000000); -- scale factor

In [0]:
-- JSON string table (one column)
DROP TABLE IF EXISTS json_str;
CREATE TABLE json_str AS
SELECT payload_json AS payload
FROM v_payload_json;

-- VARIANT table (one column) — same content, parsed once
DROP TABLE IF EXISTS json_variant;
CREATE TABLE json_variant AS
SELECT parse_json(payload_json) AS payload
FROM v_payload_json;


In [0]:
OPTIMIZE json_str;
OPTIMIZE json_variant;

In [0]:
DESCRIBE DETAIL json_str;
DESCRIBE DETAIL json_variant;

In [0]:
SET use_cached_result = false;  

Filter on a nested

In [0]:
-- JSON string
SELECT count(*) AS users_25_35
FROM json_str
WHERE payload:user.age::int BETWEEN 25 AND 35;


In [0]:
-- VARIANT
SELECT count(*) AS users_25_35
FROM json_variant
WHERE payload:user.age::int BETWEEN 25 AND 35;

Group by a nested string (country), aggregate nested int (clicks)

In [0]:
-- JSON string
CREATE OR REPLACE TABLE agg_country_clicks AS
SELECT payload:user.country::string AS country,
       SUM(payload:metrics.clicks::int) AS clicks
FROM json_str
GROUP BY country
ORDER BY clicks DESC;

In [0]:
-- VARIANT
CREATE OR REPLACE TABLE agg_country_clicks_variant AS
SELECT payload:user.country::string AS country,
       SUM(payload:metrics.clicks::int) AS clicks
FROM json_variant
GROUP BY country
ORDER BY clicks DESC;

Top‑N by a nested numeric

In [0]:
-- JSON string
CREATE OR REPLACE TABLE top_users_spent AS
SELECT payload:user.id::string AS user_id,
       payload:metrics.spent::double AS spent
FROM json_str
ORDER BY spent DESC;

In [0]:
-- VARIANT
CREATE OR REPLACE TABLE top_users_spent_variant AS
SELECT payload:user.id::string AS user_id,
       payload:metrics.spent::double AS spent
FROM json_variant
ORDER BY spent DESC;